In [6]:
from models.vgg11_yolo import vgg11_yolo as used_model
import config
import torch

folder = "../runs/VGG/dropout-0/2023-22-11_15-55-54"
path_to_model = folder + "/model.pt"
path_to_export = folder + "/model.onnx"
dropout = 0

cfg = config.Config().replace(DROPOUT=dropout, BATCH_SIZE=8)
model = used_model(split_size=cfg.SPLIT_SIZE, num_boxes=cfg.NUM_BOXES, num_classes=cfg.NUM_CLASSES, dropout_percentage=dropout).to(cfg.DEVICE)
model.load_state_dict(torch.load(path_to_model, map_location=torch.device(cfg.DEVICE)))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [7]:
import torchinfo
torchinfo.summary(model, input_size=(1, 3, 448, 448))

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 539]                  --
├─Sequential: 1-1                        [1, 512, 14, 14]          --
│    └─Conv2d: 2-1                       [1, 64, 448, 448]         1,792
│    └─BatchNorm2d: 2-2                  [1, 64, 448, 448]         128
│    └─ReLU: 2-3                         [1, 64, 448, 448]         --
│    └─MaxPool2d: 2-4                    [1, 64, 224, 224]         --
│    └─Conv2d: 2-5                       [1, 128, 224, 224]        73,856
│    └─BatchNorm2d: 2-6                  [1, 128, 224, 224]        256
│    └─ReLU: 2-7                         [1, 128, 224, 224]        --
│    └─MaxPool2d: 2-8                    [1, 128, 112, 112]        --
│    └─Conv2d: 2-9                       [1, 256, 112, 112]        295,168
│    └─BatchNorm2d: 2-10                 [1, 256, 112, 112]        512
│    └─ReLU: 2-11                        [1, 256, 112, 112]        --


In [2]:
x = torch.randn(1, 3, 448, 448)
torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  path_to_export,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  )

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

